In [ ]:
#instalaçao das bibliotecas (opicional)
#import pip
#pip.main(["install","vscode-ipynb-py-convert"])


In [ ]:
#Pandas: Possui inúmeras funções e comandos para importar arquivos, analisar dados, tratar dados, etc.
import pandas as pd # importando a biblioteca pandas

#Numpy: Possui uma série de funções e comandos para trabalharmos com números de forma em geral(formatação, calculos, etc)
import numpy as np # importando a biblioteca numpy

#Time: Possui uma série de funções e comandos para trabalharmos com tempo
import time # importando a biblioteca time

#glob: Possui uma série de funções e comandos para trabalharmos com arquivos
import glob # importando a biblioteca glob

#os: Possui uma série de funções e comandos para trabalharmos com arquivos
import os # importando a biblioteca os

#XlsxWriter: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import xlsxwriter # importando a biblioteca xlsxwriter

#Pyexcel: Possui uma série de funções e comandos para trabalharmos com arquivos excel
import pyexcel as pe # importando a biblioteca pyexcel

#Warnings: Possui detalhes sobre os avisos e alertas que aparecem, porém podemos utiliza-lo também para que os alertas de
#futuras atualizações e metodos depreciados não sejam exibidos
import warnings
warnings.filterwarnings("ignore") 
tempo_inicial = time.time() # tempo inicial para calcular o tempo de execução do código

from glob import glob # Utilizado para listar arquivos de um diretório

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Colorindo o texto
class colors:
    OK = '\033[92m'
    ERRO = '\033[91m'
    ALERTA = '\033[43m'
    NORMAL = '\033[0m'

## Importação dos Dados do CNES 

In [ ]:
df_cnes_leitos = pd.read_csv('BASE\.BASE_CNES_LEITOS.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_leitos.head()

In [ ]:
df_cnes_habilitacao = pd.read_csv('BASE\.BASE_CNES_HABILITACAO.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_habilitacao.head()

In [ ]:
df_cnes_servicos = pd.read_csv('BASE\.BASE_CNES_SERVICOS.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_servicos.head()

## Importação dos Dados do SIGTAP

In [ ]:
df_sigtap = pd.read_csv('BASE\.BASE_SIGTAP_GERAL.csv', sep=';', encoding='latin-1', dtype=str)
df_sigtap['COD_PROCEDIMENTO']= df_sigtap['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_sigtap['QT_PROD AIH 2018 a 2022']= df_sigtap['QT_PROD AIH 2018 a 2022'].astype(int) # Converte a coluna 'QT_PROD AIH 2018 a 2022' para string
df_sigtap.head() # Exibe as 5 primeiras linhas do arquivo

## Importação TETO MAC

In [ ]:
df_teto = pd.read_csv('BASE\.BASE_TETO_MAC.csv', sep=';', encoding='latin-1', dtype=str)
df_teto.head()

## Importação da PLANILHA  

### Analise e tratamento da PLANILHA ABA 1 

In [ ]:
df_planilha = glob('PLANILHA\*.xlsx')[0] # Planilha para ser validada

In [ ]:
df_planilha_aba1 = pd.read_excel(df_planilha, sheet_name='Ident. Fila na UF') # Lê o arquivo excel
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
df_planilha_aba1.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA':'COD_PROCEDIMENTO','Unnamed: 1':'DESC_PROCEDIMENTO','Unnamed: 2':'QUANT_FILA', 
                                 'Unnamed: 3':'PERC_REDUCAO', 'Unnamed: 4':'TEMPO_MESES', 'Unnamed: 5':'QUANT_REDUCAO', 'Unnamed: 6':'LINHA'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'
df_planilha_aba1.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba1.drop(1, inplace=True) # Remove a segunda linha do arquivo
df_planilha_aba1.drop(df_planilha_aba1.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba1.dropna(subset=['COD_PROCEDIMENTO'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba1.drop('Unnamed: 7', axis=1, inplace=True) # Removendo coluna 'Unnamed: 7'
df_planilha_aba1['COD_PROCEDIMENTO'] = df_planilha_aba1['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_planilha_aba1['PERC_REDUCAO'] = df_planilha_aba1['PERC_REDUCAO'].astype(float) # Converte a coluna 'QUANT_FILA' para string
df_planilha_aba1['QUANT_REDUCAO'] = df_planilha_aba1['QUANT_REDUCAO'].astype(int) # Converte a coluna 'QUANT_REDUCAO' para string
df_planilha_aba1['QUANT_FILA'] = df_planilha_aba1['QUANT_FILA'].astype(int) # Converte a coluna 'QUANT_FILA' para string

df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

### Verificar procedimento duplicado

In [ ]:
# Verifica se há valores duplicados na coluna "COD_PROCEDIMENTO"
df_planilha_aba1['DUPLICADO'] = df_planilha_aba1['COD_PROCEDIMENTO'].duplicated(keep=False)

# Cria uma nova coluna com as mensagens correspondentes
df_planilha_aba1['M_DUPLICADO'] = np.where(df_planilha_aba1['DUPLICADO'], 'SIM', 'NÃO')
df_planilha_aba1.drop('DUPLICADO', axis=1, inplace=True) # Remove a coluna 'DUPLICADO'
# Exibe as 5 primeiras linhas do arquivo com a nova coluna
df_planilha_aba1.head()

### Verificar se existe atendimento na Quantidade de Reduçao numero inferior a 1 procedimento. 

In [ ]:
# Adicionando nova coluna 'QUANT_ZERO' com a mensagem correspondente

def mensagem(quant_zero):
    if quant_zero ['QUANT_REDUCAO'] < 1 and quant_zero['PERC_REDUCAO'] > 0 :
        return 'quant. Menor que 1'
    else:
        return 'quant. OK!'
df_planilha_aba1['QUANT_ZERO'] = df_planilha_aba1.apply(mensagem, axis=1)

# Exibindo as primeiras linhas do dataframe com a nova coluna
df_planilha_aba1.head() 


#### Procedimento requer habilitação

In [ ]:
df_sigtap_h = df_sigtap[['COD_PROCEDIMENTO','EXIGE HABILITACAO','CO_HABILITACAO']] # Cria um novo dataframe com as colunas 'COD_PROCEDIMENTO','EXIGE HABILITACAO','CO_HABILITACAO'
df_sigtap_h.drop_duplicates(subset='COD_PROCEDIMENTO', keep='first', inplace=True) # Remove os valores duplicados da coluna 'COD_PROCEDIMENTO'
df_planilha_aba1['PROC_HABILITACAO'] = df_planilha_aba1['COD_PROCEDIMENTO'].map(df_sigtap_h.set_index('COD_PROCEDIMENTO')['EXIGE HABILITACAO']) # Adiciona uma nova coluna com a informação de habilitação do procedimento
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

### Analise e tratamento da PLANILHA ABA 2

In [ ]:
df_planilha_aba2 = pd.read_excel(df_planilha, sheet_name='Ident. CNES e Proced.', dtype=str) # Lê o arquivo excel
df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
# Tratamento dos dados da aba 'Ident. CNES e Proced.'
df_planilha_aba2.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES':'CNES','Unnamed: 1':'ESTABELECIMENTO','Unnamed: 2':'COD_PROCEDIMENTO', 
                                 'Unnamed: 3':'DES_PROCEDIMENTO', 'Unnamed: 4':'COMPL_RECURSO_FEDERAL', 'Unnamed: 5':'GESTAO', 'Unnamed: 6':'COD_NATUREZA','Unnamed: 7':'NATUREZA',
                                 'Unnamed: 8':'POSSUI_CONTRATO','Unnamed: 9':'IDENTIFICACAO','Unnamed: 10':'LINHA'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'
df_planilha_aba2.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba2.drop(1, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba2.drop(df_planilha_aba2.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba2.dropna(subset=['CNES'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba2.drop('IDENTIFICACAO', axis=1, inplace=True) # Removendo coluna 'IDENTIFICACAO'
df_planilha_aba2['COD_PROCEDIMENTO'] = df_planilha_aba2['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_cnes_habilitacao = df_cnes_habilitacao.rename(columns={"CO_CNES": "CNES"}) # Renomeia a coluna 'CO_CNES' para 'CNES'
df_planilha_aba2 = df_planilha_aba2.merge(df_cnes_habilitacao[["CNES", "CO_MUNICIPIO_GESTOR"]], on="CNES", how="left") # Adiciona a coluna 'CO_MUNICIPIO_GESTOR' ao dataframe

df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
df_planilha_aba2.info() 

#### Verificar se o CNES esta ATIVO:

In [ ]:
df_cnes_habilitacao['CNES'] = df_cnes_habilitacao['CNES'].astype(str) # Converte a coluna 'CNES' para string
df_cnes_habilitacao2 = df_cnes_habilitacao.loc[df_cnes_habilitacao['CO_MOTIVO_DESAB'] > '0'] # Seleciona apenas os CNES habilitados
df_cnes_habilitacao2.head() # Exibe as 5 primeiras linhas do arquivo

df_planilha_aba2['CNES_ATIVO'] = np.where(df_planilha_aba2['CNES'].isin(df_cnes_habilitacao2['CNES']), 'NÃO', 'SIM') # Adiciona a coluna 'CNES_ATIVO' ao dataframe
df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

#### Verificar se existe prestador sem procedimento informado em fila (relação ABA1 E ABA2)

In [ ]:
# Idenfica se o procedimento informado na aba anterior está na aba atual
df_planilha_aba2['PROC_INFORMADO'] = np.where(df_planilha_aba2['COD_PROCEDIMENTO'].isin(df_planilha_aba1['COD_PROCEDIMENTO']), 'SIM','NÃO')     
df_planilha_aba2.head()                                

#### Verificar se existe procedimento sem prestador informado (relação ABA2 e ABA1)

In [ ]:
# Verifica se o procedimento informado possui prestador informado
df_planilha_aba1['POSSUI_PRESTADOR'] = np.where(df_planilha_aba1['COD_PROCEDIMENTO'].isin(df_planilha_aba2['COD_PROCEDIMENTO']),'SIM','NÃO')
df_planilha_aba1.head()

#### Verificar se o procedimento informado é valido 

In [ ]:
df_planilha_aba1['PROC_VALIDO'] = np.where(df_planilha_aba1['COD_PROCEDIMENTO'].isin(df_sigtap['COD_PROCEDIMENTO']), 'SIM','NÃO')
df_planilha_aba1.head()

#### Verificar habilitação x CNES

### Analise e tratamento da PLANILHA ABA 3

In [ ]:
df_planilha_aba3 = pd.read_excel(df_planilha, sheet_name='Execução') # Lê o arquivo excel
df_planilha_aba3.head(10) # Exibe as 5 primeiras linhas do arquivo

In [ ]:
valor_portaria = df_planilha_aba3.iloc[0,3] # Armazena o valor da portaria
df_planilha_aba3.drop(df_planilha_aba3.index[0:6], inplace=True) # Remove as 5 primeiras linhas do arquivo
df_planilha_aba3.rename(columns={'Distribuição e Cronograma da Execução do Recurso Financeiro':'CODIGO GESTOR','Unnamed: 1':'GESTÃO','Unnamed: 2':'DESC_GESTOR','Unnamed: 3':'VALOR','Unnamed: 4':'MARÇO',
                                 'Unnamed: 5':'ABRIL','Unnamed: 6':'MAIO','Unnamed: 7':'JUNHO','Unnamed: 8':'JULHO','Unnamed: 9':'AGOSTO','Unnamed: 10':'SETEMBRO','Unnamed: 11':'OUTUBRO',
                                 'Unnamed: 12':'NOVEMBRO','Unnamed: 13':'DEZEMBRO','Unnamed: 14':'TOTAL_%','SQ (CODIGO Interno':'LINHA'}, inplace=True)
df_planilha_aba3.drop(df_planilha_aba3.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba3.dropna(subset=['CODIGO GESTOR'], inplace=True) # Removendo linhas com valores nulos
valor_total = df_planilha_aba3['VALOR'].sum() # Soma o valor total da coluna 'VALOR'
df_planilha_aba3['VALOR'] = df_planilha_aba3['VALOR'].astype(float) # Converte a coluna 'VALOR' para float
df_planilha_aba3['VALOR_R'] = df_planilha_aba3['VALOR'].apply(lambda x: 'R$ ' + format(x, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',')) # Formata a coluna 'VALOR' para moeda

df_planilha_aba3.head() # Exibe as 5 primeiras linhas do arquivo

In [ ]:
df_planilha_aba3['CODIGO GESTOR'] = df_planilha_aba3['CODIGO GESTOR'].astype(int) # Converte a coluna 'MUNIC_PRESTADOR' para string
df_planilha_aba3['MUNIC_PRESTADOR'] = np.where(df_planilha_aba3['CODIGO GESTOR'].isin(df_planilha_aba2['CO_MUNICIPIO_GESTOR']), 'SIM','NÃO')
df_planilha_aba3_m = df_planilha_aba3[df_planilha_aba3['GESTÃO'] == 'MUNICIPAL'] # Filtra os dados da aba 3 que possuem gestão municipal
df_teto['Código IBGE'] = df_teto['Código IBGE'].astype(int) # Converte a coluna 'Código IBGE' para string
df_planilha_aba3 = pd.merge(df_planilha_aba3, df_teto, left_on='CODIGO GESTOR', right_on='Código IBGE', how='left') # Realiza o merge das duas planilhas

df_planilha_aba3.drop(['Código IBGE', 'Código Gestão', 'Descrição Gestão'], axis=1, inplace=True) # Remove a coluna 'Código IBGE'
df_planilha_aba3.rename(columns={'Descrição Gestão':'DESC_GESTOR','Estado / Município':'DESC_GESTOR_C','Teto Financeiro MAC - Valores Anuais (R$)':'TETO_FINANC_ANO'}, inplace=True) # Renomeia a coluna 'Descrição Gestão'
df_planilha_aba3['TETO_FINANC_ANO'] = df_planilha_aba3['TETO_FINANC_ANO'].astype(float) # Converte a coluna 'TETO_FINANC_ANO' para float
df_planilha_aba3['TETO_FINANC_MES'] = df_planilha_aba3['TETO_FINANC_ANO'] / 12 # Calcula o teto financeiro mensal

df_planilha_aba3.head()


In [ ]:
diferenca = valor_portaria - valor_total # Calcula a diferença entre o valor da portaria e o valor total
diferenca = 'R$ ' + format(diferenca, ',.2f').replace('.', '#').replace(',', '.').replace('#', ',') # Formata a diferença para moeda
print(diferenca)

# SALVANDO OS RESULTADOS    

In [ ]:
df_planilha = os.path.splitext(os.path.basename(df_planilha))[0] # Pega o nome do arquivo sem a extensão
file_nome = df_planilha.split('/')[-1] # Pega o nome do arquivo
writer = pd.ExcelWriter(f'PLANILHA/{file_nome}_resultado.xlsx', engine='xlsxwriter') # Cria um arquivo excel

df_planilha_aba1.to_excel(writer, sheet_name='Aba 1', index=False)
df_planilha_aba2.to_excel(writer, sheet_name='Aba 2', index=False)
df_planilha_aba3.to_excel(writer, sheet_name='Aba 3', index=False)

writer.save()


# RELATORIO FINAL (ANALISE)

In [ ]:
# Verificação de procedimentos duplicados
if df_planilha_aba1['M_DUPLICADO'].str.contains('SIM').any():
    print(f"{colors.ERRO}[ERRO] - ABA 1 - Existem procedimentos na Fila, duplicado;")
    procedimento_duplicado = True
else:
    print(f"{colors.OK}[OK] - ABA 1 - Não existem procedimentos duplicados;")
    procedimento_duplicado = False


# VerificaçãO de procedimentos com quantidade menor que 1
if df_planilha_aba1['QUANT_ZERO'].str.contains('quant. Menor que 1').any():   
    print(f"{colors.ERRO}[ERRO] - ABA 1 - Existem procedimentos na Fila, cuja quantidade de cirurgia ficou inferior a 1 procedimento (PACIENTE);")
    quant_menor_1 = True
else:
    print(f"{colors.OK}[OK] - ABA 1 - Não existem procedimentos com quantidade menor que 1;")
    quant_menor_1 = False


# Verificação de procedimentos inválidos
if df_planilha_aba1['PROC_VALIDO'].str.contains('NÃO').any():
    print(f"{colors.ERRO}[ERRO] - ABA 1 - Existem procedimentos na Fila, que não são válidos;")
    procedimento_invalido = True
else:
    print(f"{colors.OK}[OK] - ABA 1 - Não existem procedimentos inválidos;")
    procedimento_invalido = False    


# Verificação de procedimentos sem prestador
if df_planilha_aba1['POSSUI_PRESTADOR'].str.contains('NÃO').any():
    print(f"{colors.ERRO}[ERRO] - ABA 1/2 - Existem procedimentos na Fila, cuja não existe prestador;")
    sem_prestador = True
else:
    print(f"{colors.OK}[OK] - ABA 1/2 - Não existem procedimentos sem prestador;")
    sem_prestador = False


# Verificação de CNES ativo
if df_planilha_aba2['CNES_ATIVO'].str.contains('SIM').any():
    print(f"{colors.ERRO}[ERRO] - ABA 2 - Existem CNES inativos;")
    cnes_inativo = True
else:
    print(f"{colors.OK}[OK] - ABA 2 - Não existem CNES inativos;")
    cnes_inativo = False


# Verificar valor de portaria
if valor_total == valor_portaria:
    print(f"{colors.OK}[OK] - ABA 3 - Valor programado igual ao valor alocado na Portaria 90;")
    valor_portaria_correto = True
else:   
    print(f"{colors.ERRO}[ERRO] - ABA 3 - Valor programado diferente do valor alocado na Portaria 90; {diferenca}")
    valor_portaria_correto = False
    

# Verificar se existe municipio com repasse, mas não existe CNES relacionado ao municipio
if df_planilha_aba3_m['MUNIC_PRESTADOR'].str.contains('NÃO').any():   
    print(f"{colors.ERRO}[ERRO] - ABA 3 - Existem municipio com repasse, mas não existe CNES relacionado ao municipio;")
    
    sem_prestador = True
else:   
    print(f"{colors.OK}[OK] - ABA 3 - Relação de municipios diacordo com o CNES;")
    sem_prestador = False

# Arquivo gerado com os resultados
print(f"{colors.OK}[OK] - Arquivo: '{file_nome}- resultado.xlsx' gerado com sucesso!")






In [ ]:
df_planilha_aba1.info()

In [ ]:
df_planilha_aba2.head()

In [ ]:
df_planilha_aba3.head()

In [ ]:
df_planilha_aba1 xxxxxx

In [ ]:
df_planilha = os.path.splitext(os.path.basename(df_planilha))[0] # Pega o nome do arquivo sem a extensão
file_nome = df_planilha.split('/')[-1] # Pega o nome do arquivo
print(file_nome)

In [ ]:
df_juncao2

In [ ]:
df_juncao1

In [ ]:
# Adicionando nova coluna 'QUANT_ZERO' com a mensagem correspondente
def mensagem(quant_zero):
    if quant_zero ['PERC_REDUCAO'] > 0 and quant_zero['QUANT_REDUCAO'] < 1:
        return 'quant. Menor que 1'
    else:
        return 'quant. OK!'
df_planilha_aba1['QUANT_ZERO'] = df_planilha_aba1.apply(mensagem, axis=1)

# Exibindo as primeiras linhas do dataframe com a nova coluna
df_planilha_aba1.head()


In [ ]:
if df_planilha_aba1['PERC_REDUCAO'].float() < 0:
    print(f"{colors.ERRO}[ERRO] - Existem procedimentos na Fila, com redução de quantidade;")
    procedimento_duplicado = True

In [ ]:
# Gerando arquivo de saída (relatório) - excel

In [ ]:
if df_planilha_aba1['QUANT_ZERO'].str.contains('quant. Menor que 1').any():
    
    
    print(f"{colors.ERRO}[ERRO] - Existem procedimentos na Fila, cuja quantidade de cirurgia ficou inferior a 1 procedimento (PACIENTE);")
    quant_menor_1 = True
else:
    print(f"{colors.OK}[OK] - Não existem procedimentos com quantidade menor que 1;")
    quant_menor_1 = False

In [ ]:
# Verifica se os valores da coluna "COD_PROCEDIMENTO" do DataFrame "df_planilha_aba2" estão contidos na coluna "COD_PROCEDIMENTO" do DataFrame "df_planilha_aba1"
df_planilha_aba2['PROC_INFORMADO'] = np.where(df_planilha_aba2['COD_PROCEDIMENTO'].isin(df_planilha_aba1['COD_PROCEDIMENTO']), 'proc. Informado na Ident. Fila na UF', 'proc. não informado na aba anterior')
df_planilha_aba2.head()

In [ ]:
merged = df_planilha_aba2.merge(df_planilha_aba1[['COD_PROCEDIMENTO']], on='COD_PROCEDIMENTO', how='left', indicator='PROC_INFORMADO')
df_planilha_aba2['PROC_INFORMADO'] = np.where(merged['PROC_INFORMADO'] == 'both', 'proc. Informado na Ident. Fila na UF', 'proc. não informado na aba anterior')



In [ ]:
df_planilha_aba2.head(315)

In [ ]:
df_planilha_aba1.head(315)


In [ ]:
tempo_final = time.time()
tempo_total = (float(tempo_final - tempo_inicial)/60)
print(f"Tempo total de execução: {tempo_total:,.1f} minutos" )

In [ ]:
df_planilha_aba2['COD_PROCEDIMENTO'] = df_planilha_aba2['COD_PROCEDIMENTO'].astype(int) # Converte a coluna 'COD_PROCEDIMENTO' para string

In [ ]:
# Fazer a junção entre df_planilha_aba2 e df_sigtap usando a coluna COD_PROCEDIMENTO
df_sigtap['CO_PROCEDIMENTO'] = df_sigtap['CO_PROCEDIMENTO'].astype(int) # Converte a coluna 'CO_PROCEDIMENTO' para string
df_juncao1 = pd.merge(df_planilha_aba2, df_sigtap, on="COD_PROCEDIMENTO", how="left")

# Criar uma nova coluna CO_HABILITACAO_OK que armazena o código da habilitação se EXIGE HABILITACAO for SIM, caso contrário passa
df_juncao1["CO_HABILITACAO_OK"] = df_juncao1.apply(lambda row: row["CO_HABILITACAO"] if row["EXIGE HABILITACAO"] == "SIM" else None, axis=1)

# Fazer a junção entre df_juncao1 e df_cnes_habilitacao usando as colunas CNES e CO_HABILITACAO_OK
df_juncao2 = pd.merge(df_juncao1, df_cnes_habilitacao, left_on=["CNES", "CO_HABILITACAO_OK"], right_on=["CNES", "CO_CODIGO_GRUPO"], how="left")

# Criar uma nova coluna UNID_HABILITACAO que armazena a mensagem "unidade ok" se CO_CODIGO_GRUPO for igual a CO_HABILITACAO_OK, caso contrário armazena "unidade sem habilitação"
df_juncao2["UNID_HABILITACAO"] = df_juncao2.apply(lambda row: "UND_OK" if row["CO_CODIGO_GRUPO"] == row["CO_HABILITACAO_OK"] else "unidade sem habilitação", axis=1)


